# Data

### Import Required Libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
import os
print(os.getcwd())

### Load Connectivity Data

In [2]:
# Load connectivity matrix from CSV
connect_matrix = pd.read_csv("hexFlow.csv", sep=",",index_col=0)
print(connect_matrix)

            1         2         3         4         5         6         7  \
1    0.000033  0.000033  1.088253  0.272063  0.120917  0.068016  0.000033   
2    0.000028  0.000028  0.000028  0.919358  0.229840  0.102151  2.758201   
3    0.871399  0.000026  0.000026  0.000026  0.871399  0.217850  0.373455   
4    0.222155  0.888621  0.000027  0.000027  0.000027  0.888621  0.140307   
5    0.109651  0.246715  0.986860  0.000030  0.000030  0.000030  0.080017   
..        ...       ...       ...       ...       ...       ...       ...   
649  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
650  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
651  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
652  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
653  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

            8         9        10  ...       644       645       646  \
1  

In [ ]:
# Remove 1st column because it sesrves as an index and it's not needed
connect_matrix = connect_matrix.iloc[:, 1:]
print(connect_matrix)

# Transition Matrix

### Normalize

In [3]:
# Function to normalize rows
def normalize_rows(df):
    row_sums = df.sum(axis=1)
    normalized_df = df.div(row_sums, axis=0)
    return normalized_df

# Normalize the connectivity matrix
nor_matrix = normalize_rows(connect_matrix)

#### Verifications of Matrix

In [4]:
# Verify that each row sums to 1
row_sums = nor_matrix.sum(axis=1)
print(row_sums)  # Should print a Series of 1s

# Dimensions
print(nor_matrix.shape)

# Non-Negative Values
print(nor_matrix.min().min())

# Check if the row names and column names are the same
print(nor_matrix.index.equals(nor_matrix.columns))

1      1.0
2      1.0
3      1.0
4      1.0
5      1.0
      ... 
649    1.0
650    1.0
651    1.0
652    1.0
653    1.0
Length: 653, dtype: float64
(653, 653)
0.0
False


# Initial states

### Data

In [5]:
hex_planning_units = gpd.read_file("hex_planning_units.shp")

# Renaming 'FID' to 'id' and incrementing by 1
hex_planning_units = hex_planning_units.rename(columns={'FID': 'id'})
hex_planning_units['id'] = hex_planning_units['id'].astype(int) + 1

### Initial states list

In [6]:
# Get the names of all columns that start with "BIORE"
biore_columns = [col for col in hex_planning_units.columns if col.startswith("BIORE")]

# Create a dictionary to store the normalized initial states for each BIORE column
initial_states = {}

# Loop through each BIORE column
for biore_col in biore_columns:
    
    # Extract the column into a vector and handle NA values, ensuring non-negative entries
    biore_vector = hex_planning_units[biore_col].fillna(0).clip(lower=0)
    
    # Transform the vector into a row vector
    biore_row = biore_vector.values.reshape(1, -1)
    
    # Ensure the row vector's column names match those of the transition matrix (nor_matrix)
    biore_row_df = pd.DataFrame(biore_row, columns=nor_matrix.columns)
    
    # Rename the row to match the column name
    biore_row_df.index = [biore_col]
    
    # Check if the initial state distribution has the correct dimensions and non-negative values
    if biore_row_df.shape[1] != nor_matrix.shape[1]:
        raise ValueError(f"The initial state distribution for {biore_col} does not have the correct number of columns.")
    if (biore_row_df < 0).any().any():
        raise ValueError(f"Some entries in the initial distribution for {biore_col} are negative.")
    
    # Normalize the initial distribution so that the sum is 1
    biore_row_normalized = biore_row_df.div(biore_row_df.sum(axis=1), axis=0)
    
    # Ensure that the normalized distribution sums to 1
    if not np.isclose(biore_row_normalized.sum(axis=1), 1).all():
        raise ValueError(f"The normalized initial state distribution for {biore_col} does not sum to 1.")
    
    # Store the normalized row vector in the initial_states dictionary
    initial_states[biore_col] = biore_row_normalized
    
    # Print a message indicating completion for each column
    print(f"Processed {biore_col} as initial state.")


Processed BIORE_102 as initial state.
Processed BIORE_103 as initial state.
Processed BIORE_114 as initial state.
Processed BIORE_116 as initial state.
Processed BIORE_117 as initial state.
Processed BIORE_118 as initial state.
Processed BIORE_119 as initial state.
Processed BIORE_121 as initial state.
Processed BIORE_13 as initial state.
Processed BIORE_132 as initial state.
Processed BIORE_142 as initial state.
Processed BIORE_147 as initial state.
Processed BIORE_15 as initial state.
Processed BIORE_16 as initial state.
Processed BIORE_23 as initial state.
Processed BIORE_26 as initial state.
Processed BIORE_27 as initial state.
Processed BIORE_29 as initial state.
Processed BIORE_3 as initial state.
Processed BIORE_4 as initial state.


### Verification of Initial States

In [ ]:
print("Structure of initial_states:")
for key, value in initial_states.items():
    print(f"{key}: shape {value.shape}")


In [ ]:
# Viewing a specific BIORE column, e.g., BIORE_147
print("Viewing initial_states for BIORE_147:")
print(initial_states['BIORE_147'])

# Printing another specific BIORE column, e.g., BIORE_102
print("Initial state distribution for BIORE_102:")
print(initial_states['BIORE_102'])


# Dynamic model

### Set up

In [7]:
# Create a dictionary to store the data frames for each feature
iterations_results = {}

# Define tolerance for steady-state convergence
tolerance = 1e-4

###  Loop Through Each Feature's Initial State

In [8]:
for feature_name, previous_state in initial_states.items():
    previous_state = previous_state.values.flatten()
    current_state = previous_state.copy()
    
    # Initialize a list to hold each step's data
    steps_data = [previous_state]  # Start with the initial state
    
    iterations = 0
    while True:
        current_state = np.dot(previous_state, nor_matrix.values)
        steps_data.append(current_state)  # Append current state to the list
        
        iterations += 1
        if np.sum(np.abs(current_state - previous_state)) < tolerance:
            break
        previous_state = current_state

    # After the loop, convert the list of numpy arrays into a DataFrame
    feature_df = pd.DataFrame(steps_data).T  # Transpose to match your original format
    feature_df.columns = [f"Step_{i}" for i in range(iterations + 1)]  # Naming columns after steps
    
    iterations_results[feature_name] = feature_df
    print(f"Processed {feature_name} in {iterations} iterations")

Processed BIORE_102 in 470 iterations
Processed BIORE_103 in 488 iterations
Processed BIORE_114 in 242 iterations
Processed BIORE_116 in 462 iterations
Processed BIORE_117 in 367 iterations
Processed BIORE_118 in 468 iterations
Processed BIORE_119 in 448 iterations
Processed BIORE_121 in 439 iterations
Processed BIORE_13 in 486 iterations
Processed BIORE_132 in 482 iterations
Processed BIORE_142 in 398 iterations
Processed BIORE_147 in 428 iterations
Processed BIORE_15 in 411 iterations
Processed BIORE_16 in 423 iterations
Processed BIORE_23 in 439 iterations
Processed BIORE_26 in 409 iterations
Processed BIORE_27 in 430 iterations
Processed BIORE_29 in 448 iterations
Processed BIORE_3 in 454 iterations
Processed BIORE_4 in 420 iterations


### Verification of steps

In [9]:
# Example: Access the DataFrame for BIORE_102
if "BIORE_102" in iterations_results:
    print("Data for BIORE_102:")
    print(iterations_results["BIORE_102"].head())  # Using .head() to print the first few rows

# Loop through the names and print each one
for feature_name, df in iterations_results.items():
    print(f"Feature data frame: {feature_name}")

Data for BIORE_102:
   Step_0  Step_1  Step_2        Step_3        Step_4        Step_5  \
0     0.0     0.0     0.0  2.464406e-09  1.854455e-08  6.618710e-08   
1     0.0     0.0     0.0  1.822032e-09  1.551079e-08  5.902603e-08   
2     0.0     0.0     0.0  1.333151e-09  1.255032e-08  5.002717e-08   
3     0.0     0.0     0.0  9.538688e-10  9.929398e-09  4.118256e-08   
4     0.0     0.0     0.0  6.373710e-10  7.555161e-09  3.248779e-08   

         Step_6        Step_7        Step_8        Step_9  ...  Step_461  \
0  1.691110e-07  3.547735e-07  6.521282e-07  1.089787e-06  ...  0.000762   
1  1.573355e-07  3.405734e-07  6.416164e-07  1.093860e-06  ...  0.000902   
2  1.374896e-07  3.045203e-07  5.843339e-07  1.011462e-06  ...  0.000952   
3  1.161304e-07  2.621655e-07  5.107394e-07  8.951575e-07  ...  0.000933   
4  9.345664e-08  2.139424e-07  4.213439e-07  7.450970e-07  ...  0.000840   

   Step_462  Step_463  Step_464  Step_465  Step_466  Step_467  Step_468  \
0  0.000762  0.000762

### Step sums Data frame

In [10]:
# Initialize DataFrame with an index that matches the expected range of IDs
final_markov = pd.DataFrame(index=np.arange(1, 654))  # np.arange starts at 1 and ends at 653 inclusive

# Loop through each BIORE feature in the iterations_results dictionary
for feature_name, biore_df in iterations_results.items():
    
    # Exclude 'Step_0' and sum the rest of the steps (Step_1, Step_2, ..., Step_N)
    steps_columns = [col for col in biore_df.columns if "Step_" in col and col != "Step_0"]

    # Create a new column 'Step_Sum' that sums all the remaining Step columns
    biore_sum = biore_df[steps_columns].sum(axis=1)

    # Add this sum as a new column in the final_markov DataFrame, with the column named after the BIORE feature
    final_markov[feature_name] = biore_sum

    # Print a message indicating the sum is computed for this feature
    print(f"Computed sum for feature: {feature_name}")

# Add the 'id' column with values from 1 to 653
final_markov['id'] = np.arange(1, 654)

# Print the first few rows of the final_markov DataFrame
print("First few rows of the final_markov DataFrame:")
print(final_markov.head())


Computed sum for feature: BIORE_102
Computed sum for feature: BIORE_103
Computed sum for feature: BIORE_114
Computed sum for feature: BIORE_116
Computed sum for feature: BIORE_117
Computed sum for feature: BIORE_118
Computed sum for feature: BIORE_119
Computed sum for feature: BIORE_121
Computed sum for feature: BIORE_13
Computed sum for feature: BIORE_132
Computed sum for feature: BIORE_142
Computed sum for feature: BIORE_147
Computed sum for feature: BIORE_15
Computed sum for feature: BIORE_16
Computed sum for feature: BIORE_23
Computed sum for feature: BIORE_26
Computed sum for feature: BIORE_27
Computed sum for feature: BIORE_29
Computed sum for feature: BIORE_3
Computed sum for feature: BIORE_4
First few rows of the final_markov DataFrame:
   BIORE_102  BIORE_103  BIORE_114  BIORE_116  BIORE_117  BIORE_118  \
1   0.315301   0.316870   0.211033   0.313959   0.275610   0.314749   
2   0.332304   0.333958   0.222250   0.330887   0.290428   0.331721   
3   0.325614   0.327236   0.2176

# Results

### Rasterize

In [12]:
def create_multilayer_raster(final_markov, hex_planning_units):
    resolution = 1000
    bounds = hex_planning_units.total_bounds
    height = int((bounds[3] - bounds[1]) / resolution)
    width = int((bounds[2] - bounds[0]) / resolution)
    transform = from_origin(bounds[0], bounds[3], resolution, resolution)

    raster_dict = {}

    # Loop through each BIORE_ feature in final_markov (excluding 'id' column)
    for feature_name in final_markov.columns.drop('id'):
        print(f"Processing feature: {feature_name}")
        
        # Merge hex_planning_units with current BIORE feature data
        biore_data = final_markov[['id', feature_name]]
        hex_data = hex_planning_units[['geometry']].merge(biore_data, left_on=hex_planning_units.index, right_on='id', how='left')

        # Rasterize the summed data for the current BIORE feature
        rasterized = rasterize(
            [(geom, value) for geom, value in zip(hex_data.geometry, hex_data[feature_name])],
            out_shape=(height, width),
            fill=np.nan,
            transform=transform,
            dtype='float32'
        )

        # Normalize raster values for better visualization
        valid_mask = ~np.isnan(rasterized)
        if np.any(valid_mask):
            min_val = np.nanmin(rasterized[valid_mask])
            max_val = np.nanmax(rasterized[valid_mask])
            normalized_raster = (rasterized - min_val) / (max_val - min_val)

        raster_dict[feature_name] = normalized_raster

    return raster_dict

# Function call
raster_dict = create_multilayer_raster(final_markov, hex_planning_units)

Processing feature: BIORE_102
Processing feature: BIORE_103
Processing feature: BIORE_114
Processing feature: BIORE_116
Processing feature: BIORE_117
Processing feature: BIORE_118
Processing feature: BIORE_119
Processing feature: BIORE_121
Processing feature: BIORE_13
Processing feature: BIORE_132
Processing feature: BIORE_142
Processing feature: BIORE_147
Processing feature: BIORE_15
Processing feature: BIORE_16
Processing feature: BIORE_23
Processing feature: BIORE_26
Processing feature: BIORE_27
Processing feature: BIORE_29
Processing feature: BIORE_3
Processing feature: BIORE_4


###  Dynamic PNG

In [13]:
# Create the directory for saving PNG files if it doesn't exist
output_directory = "dynamic_results"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Plotting each raster from raster_dict and saving as PNG
for feature_name, raster in raster_dict.items():
    # Create a new figure for each plot
    fig, ax = plt.subplots(figsize=(10, 8))  # Adjust the size based on your needs
    ax.set_title(feature_name)
    raster_img = ax.imshow(raster, cmap='viridis', extent=hex_planning_units.total_bounds[[0, 2, 1, 3]])
    hex_planning_units.boundary.plot(ax=ax, edgecolor='lightgray', linewidth=0.5)
    fig.colorbar(raster_img, ax=ax, label='Normalized Value')
    
    # Save the figure
    plt.tight_layout()
    fig.savefig(f"{output_directory}/{feature_name}.png")
    plt.close(fig)  # Close the plot to free up memory

print(f"All raster images have been saved in {output_directory}.")


All raster images have been saved in dynamic_results.


### Dynamic GeoTIFF

In [18]:
def save_individual_rasters_to_tif(raster_dict, hex_planning_units, output_dir='GeoTIFF'):
    # Ensure the output directory exists; if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get the bounds and resolution from hex_planning_units
    bounds = hex_planning_units.total_bounds  # [min_x, min_y, max_x, max_y]
    resolution = 1000  # Adjust resolution if needed
    transform = from_origin(bounds[0], bounds[3], resolution, resolution)  # Top-left corner and resolution

    # Loop through each raster in the dictionary and save as separate TIF files
    for layer_name, raster_data in raster_dict.items():
        # Create a unique filename for each layer
        output_file = os.path.join(output_dir, f"{layer_name}.tif")
        
        # Define the new dataset's metadata
        with rasterio.open(
            output_file, 'w', driver='GTiff',
            height=raster_data.shape[0], width=raster_data.shape[1],
            count=1, dtype=str(raster_data.dtype),
            crs=hex_planning_units.crs.to_string(),  # Use CRS from hex_planning_units
            transform=transform
        ) as new_dataset:
            # Write the raster data to the file
            new_dataset.write(raster_data, 1)
            print(f"Saved individual layer as TIF: {output_file}")

# Example function call to save each raster as separate TIF files in 'GeoTIFF' folder
save_individual_rasters_to_tif(raster_dict, hex_planning_units, 'GeoTIFF')


Saved individual layer as TIF: GeoTIFF\BIORE_102.tif
Saved individual layer as TIF: GeoTIFF\BIORE_103.tif
Saved individual layer as TIF: GeoTIFF\BIORE_114.tif
Saved individual layer as TIF: GeoTIFF\BIORE_116.tif
Saved individual layer as TIF: GeoTIFF\BIORE_117.tif
Saved individual layer as TIF: GeoTIFF\BIORE_118.tif
Saved individual layer as TIF: GeoTIFF\BIORE_119.tif
Saved individual layer as TIF: GeoTIFF\BIORE_121.tif
Saved individual layer as TIF: GeoTIFF\BIORE_13.tif
Saved individual layer as TIF: GeoTIFF\BIORE_132.tif
Saved individual layer as TIF: GeoTIFF\BIORE_142.tif
Saved individual layer as TIF: GeoTIFF\BIORE_147.tif
Saved individual layer as TIF: GeoTIFF\BIORE_15.tif
Saved individual layer as TIF: GeoTIFF\BIORE_16.tif
Saved individual layer as TIF: GeoTIFF\BIORE_23.tif
Saved individual layer as TIF: GeoTIFF\BIORE_26.tif
Saved individual layer as TIF: GeoTIFF\BIORE_27.tif
Saved individual layer as TIF: GeoTIFF\BIORE_29.tif
Saved individual layer as TIF: GeoTIFF\BIORE_3.tif
Sa